<a href="https://colab.research.google.com/github/jermeyz/tensoflow_notebooks/blob/main/TXT_ScriptGenerating_CharLevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

## Data gathering

In [2]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt

--2021-08-18 15:53:33--  https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.128, 74.125.199.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2021-08-18 15:53:33 (122 MB/s) - ‘shakespeare.txt’ saved [1115394/1115394]



In [3]:
text = open("./shakespeare.txt", "rb").read().decode("utf-8")

Create vocabulary and map each character in original document to a unique integer.

In [4]:
print("Number of characters:", len(text))
print("-" * 50)
print(text[:300])

Number of characters: 1115394
--------------------------------------------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


Create vocabulary of all unique characters.

In [9]:
vocab = sorted(set(text))
print(vocab)
ids_to_char = np.array(vocab)
print(ids_to_char)
char_to_ids = {char: char_idx for char_idx, char in enumerate(vocab)}
print(char_to_ids)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, '

Convert each character in the text to corresponding integer.

In [10]:
text_ids = np.array([char_to_ids[char] for char in text])
print(text_ids[:300])

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1 39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39
 58 46 43 56  1 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47
 57 46 12  0  0 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53
 50 60 43 42  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47
 56 57 58  6  1 63 53 59  1 49 52 53 61  1 15 39 47 59 57  1 25 39 56 41
 47 59 57  1 47 57  1 41 46 47 43 44  1 43 52 43 51 63  1 58 53  1 58 46
 43  1 54 43 53 54 50 43  8  0  0 13 50 50 10  0 35 43  1 49 52 53 61  5
 58  6  1 61 43  1 49 52 53 61  5 58  8  0  0 18 47 56 57 58  1 15 47 58
 47 64 43 52 10  0 24 43 58  1 59 57]


## Data preparation

Split a text into multiple chunk, each chunk contains `SEQ_LEN` characters.

As the task is to predict the next character, we define the input and the target for each chunk as:
- Input: Up to the last character.
- Target: Input shifted one character to the right.

In [11]:
def train_target_split(chunk):
  input_chunk = chunk[:-1]
  target_chunk = chunk[1:]
  return input_chunk, target_chunk

In [12]:
SEQ_LEN = 64
BUFFER_SIZE = 10000
samples = tf.data.Dataset.from_tensor_slices(text_ids).batch(SEQ_LEN + 1, drop_remainder=True).map(train_target_split).shuffle(BUFFER_SIZE)

**Batchify data**

In [13]:
BATCH_SIZE = 64

test_size = 0.2
num_samples = sum(1 for x in samples)
num_train_samples = int(test_size * num_samples)
ds_train = samples.take(num_train_samples).batch(BATCH_SIZE)
ds_val = samples.skip(num_train_samples).batch(BATCH_SIZE)

## Model architecture

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional

In [15]:
EMBED_DIM = 64
VOCAB_SIZE = len(vocab)

In [16]:
model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBED_DIM))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dense(VOCAB_SIZE, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          4160      
_________________________________________________________________
bidirectional (Bidirectional (None, None, 64)          24832     
_________________________________________________________________
dense (Dense)                (None, None, 65)          4225      
Total params: 33,217
Trainable params: 33,217
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["acc"])

In [18]:
history = model.fit(ds_train, epochs=10, validation_data=ds_val)

Epoch 1/10
54/54 [==============================] - 17s 223ms/step - loss: 3.4614 - acc: 0.1386 - val_loss: 3.1984 - val_acc: 0.1557
Epoch 2/10
54/54 [==============================] - 12s 208ms/step - loss: 3.0522 - acc: 0.1823 - val_loss: 2.8792 - val_acc: 0.2138
Epoch 3/10
54/54 [==============================] - 12s 210ms/step - loss: 2.6848 - acc: 0.2607 - val_loss: 2.4567 - val_acc: 0.3373
Epoch 4/10
54/54 [==============================] - 12s 208ms/step - loss: 2.2225 - acc: 0.4292 - val_loss: 1.9868 - val_acc: 0.5148
Epoch 5/10
54/54 [==============================] - 12s 208ms/step - loss: 1.7743 - acc: 0.5796 - val_loss: 1.5651 - val_acc: 0.6464
Epoch 6/10
54/54 [==============================] - 12s 211ms/step - loss: 1.3818 - acc: 0.6985 - val_loss: 1.2180 - val_acc: 0.7485
Epoch 7/10
54/54 [==============================] - 12s 211ms/step - loss: 1.0729 - acc: 0.7857 - val_loss: 0.9413 - val_acc: 0.8181
Epoch 8/10
54/54 [==============================] - 12s 212ms/step - 